#2. 데이터 확인

In [ ]:
import pandas as pd 

movies = [
  ['user1', '연평대전', 5],
  ['user1', '7번방의 선물', 4],
  ['user1', '국제시장', 4],
  ['user2', '연평대전', 5],
  ['user2', '7번방의 선물', 3],
  ['user2', '국제시장', 4],
  ['user2', '명랑', 5],
  ['user2', '국가대표', 3],
  # ['user3', '연평대전', 4],
  ['user3', '7번방의 선물', 3],
  # ['user3', '국제시장', 3],
  ['user3', '명랑', 3],
  ['user3', '국가대표', 5]
]
df_movies = pd.DataFrame(movies, columns=['userID', 'movieName', 'Rating'])
df_movies

> 위와 같이 영화별로 평점을 처리한 데이터가 존재한다. 이를 이용하여 협업 필터링을 이용한 추천 시스템을 만들어 보자

#2. 데이터 가공 및 변환

##2.1 데이터 가공

In [ ]:
df_ratings_movies = df_movies.pivot_table('Rating', index='userID', columns='movieName')
df_ratings_movies

>위와 같이 row는 영화 이름을 columns은 사용자 id를 갖는 dataFrame을 만들 수 있다.

##2.2 데이터 변환
>위와 같이 가공한 데이터에서 평가가 되지 않는 부분에 대해서는 결측치로 나타나게 된다. 이를 해결하기 위해 결측치를 0으로 대처하자

In [14]:
df_ratings_movies = df_ratings_movies.fillna(0)
df_ratings_movies

movieName,7번방의 선물,국가대표,국제시장,명랑,연평대전
userID,,,,,
user1,4.0,0.0,4.0,0.0,5.0
user2,3.0,3.0,4.0,5.0,5.0
user3,3.0,5.0,0.0,3.0,0.0


>NaN으로 값이 지정되어 있으면 유사도를 구할 수 없기 때문에 0으로 변환시켰다.

#3. 영화 간 유사도 산출

##3.1 사용자 기반 추천
> 유사도 산출을 위해 cosine_similarity()를 이용할 것이다. cosine_similarity()는 행을 기준으로 유사도를 산출하는 함수이다. 현재 dataframe의 행은 사용자ID로 이루어져 있어 유사도를 산출하게 되면 사용자의 유사도를 구할 수 있다.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(df_ratings_movies, df_ratings_movies)
user_similarity

array([[1.        , 0.76594714, 0.24238715],
       [0.76594714, 1.        , 0.64891937],
       [0.24238715, 0.64891937, 1.        ]])

>위와 같이 코사인 유사도를 이용하여 사용자별 평점 유사도를 측정할 수 있다.

In [16]:
user_based_collab = pd.DataFrame(data=user_similarity, index=df_ratings_movies.index, columns=df_ratings_movies.index)
print(user_based_collab)
print('=======================================')
print(user_based_collab['user1'].sort_values(ascending=False))

userID     user1     user2     user3
userID                              
user1   1.000000  0.765947  0.242387
user2   0.765947  1.000000  0.648919
user3   0.242387  0.648919  1.000000
userID
user1    1.000000
user2    0.765947
user3    0.242387
Name: user1, dtype: float64


>이를 간단하게 DataFrame으로 만들고 user1과 유사도가 높은 순으로 정렬하여 출력했다. user1과 영화적 취향이 가장 비슷한 사람은 'user2'이다. 따라서 user1에게 user2가 본 영화를 추천하면 좋을 것이다.

In [17]:
df_ratings_movies.loc['user2'].sort_values(ascending=False)

movieName
연평대전       5.0
명랑         5.0
국제시장       4.0
국가대표       3.0
7번방의 선물    3.0
Name: user2, dtype: float64

> user2와 영화적 취향이 비슷하다고 판단하여 user2가 본 영화의 정렬된 정보를 확인했다. 이를 간단하게 함수를 만들어 보자.

In [8]:
def UserBaseRecommand(dataFrame, userId):
  user_similarity = cosine_similarity(dataFrame, dataFrame)
  user_based_collab = pd.DataFrame(data=user_similarity, index=dataFrame.index, columns=dataFrame.index)
  users = user_based_collab[userId].sort_values(ascending=False)
  return dataFrame.loc[users.index[1]].sort_values(ascending=False)

recommand_movies = UserBaseRecommand(df_ratings_movies, 'user3')
recommand_movies

movieName
연평대전       5.0
명랑         5.0
국제시장       4.0
국가대표       3.0
7번방의 선물    3.0
Name: user2, dtype: float64

>위와 같이 영화적 취향이 비슷한 사람의 영화를 보여주고 있다. 이번에는 사용자가 못본 영화를 추천하는 시스템을 만들어 보자.

In [11]:
df_ratings_movies_T = df_ratings_movies.transpose()
df_ratings_movies_T

userID,user1,user2,user3
movieName,,,
7번방의 선물,4.0,3.0,3.0
국가대표,0.0,3.0,5.0
국제시장,4.0,4.0,0.0
명랑,0.0,5.0,3.0
연평대전,5.0,5.0,0.0


In [9]:
df_ratings_movies_T = df_ratings_movies.transpose()
df_ratings_movies_T[
  (df_ratings_movies_T['user1']==0)
]

userID,user1,user2,user3
movieName,,,
국가대표,0.0,3.0,5.0
명랑,0.0,5.0,3.0


>안본 영화의 평점을 0으로 수정했음으로 사용자의 평점이 0과 같은 정보를 추출하면 된다. 하지만 현재 columns 정보가 영화 정보로 되어 있어 정보를 비교할 수 없다. transpose가 전치를 해 주는 함수이므로 이를 이용하여 치환하고 평점이 0인 값만 출력했다.

In [13]:
df_user = df_ratings_movies_T[
  (df_ratings_movies_T['user1']==0)
]
df_user

userID,user1,user2,user3
movieName,,,
국가대표,0.0,3.0,5.0
명랑,0.0,5.0,3.0


In [ ]:
df_user = df_ratings_movies_T[
  (df_ratings_movies_T['user1']==0)
]
df_user['user2'].sort_values(ascending=False)

> 이렇게 구한 테이블에서 사용자와 취향이 비슷한 사용자의 정보 중 보지 않은 정보를 추천할 수도 있다. 이를 간단하게 함수로 만들어 보자.

In [ ]:
def UserBaseRecommand(dataFrame, userId):
  user_similarity = cosine_similarity(dataFrame, dataFrame)
  user_based_collab = pd.DataFrame(data=user_similarity, index=dataFrame.index, columns=dataFrame.index)
  users = user_based_collab[userId].sort_values(ascending=False)

  dataFrame_t = dataFrame.transpose()
  non_view_movie=dataFrame_t[dataFrame_t[userId]==0]
  return non_view_movie[users.index[1]].sort_values(ascending=False)

recommand_movies = UserBaseRecommand(df_ratings_movies, 'user3')
recommand_movies

##3.2 아이템 기반 추천
> 기존의 DataFrame은 사용자를 행으로 가지고 있다. 아이템 기반 추천을 위해 행을 영화 제목으로 변경하고 시스템을 구축해 보자.

In [ ]:
df_ratings_movies_T = df_ratings_movies.transpose()
df_ratings_movies_T

userID,user1,user2,user3
movieName,,,
7번방의 선물,4.0,3.0,3.0
국가대표,0.0,3.0,5.0
국제시장,4.0,4.0,0.0
명랑,0.0,5.0,3.0
연평대전,5.0,5.0,0.0


>위와 같이 transpose() 함수를 이용하여 간단하게 행, 열을 치환할 수 있다.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

movies_similarity = cosine_similarity(df_ratings_movies_T, df_ratings_movies_T)
movies_similarity

array([[1.        , 0.70588235, 0.84887469, 0.70588235, 0.84887469],
       [0.70588235, 1.        , 0.36380344, 0.88235294, 0.36380344],
       [0.84887469, 0.36380344, 1.        , 0.60633906, 1.        ],
       [0.70588235, 0.88235294, 0.60633906, 1.        , 0.60633906],
       [0.84887469, 0.36380344, 1.        , 0.60633906, 1.        ]])

>각 영화별 평점 유사도를 구한 결과이다. 

In [ ]:
item_based_collab = pd.DataFrame(data=movies_similarity, index=df_ratings_movies.columns, columns=df_ratings_movies.columns)
item_based_collab

movieName,7번방의 선물,국가대표,국제시장,명랑,연평대전
movieName,,,,,
7번방의 선물,1.000000,0.705882,0.848875,0.705882,0.848875
국가대표,0.705882,1.000000,0.363803,0.882353,0.363803
국제시장,0.848875,0.363803,1.000000,0.606339,1.000000
명랑,0.705882,0.882353,0.606339,1.000000,0.606339
연평대전,0.848875,0.363803,1.000000,0.606339,1.000000


>각 영화별 평점에 따른 유사도를 dataframe으로 표현해 봤다.

In [ ]:
item_based_collab['7번방의 선물'].sort_values(ascending=False)[1:]

movieName
연평대전    0.848875
국제시장    0.848875
명랑      0.705882
국가대표    0.705882
Name: 7번방의 선물, dtype: float64

> 7번방의 선물과 연관성이 높은 영화 순으로 나열해 보니 연평대전을 추천해 주고 있다. [1:]을 이용하여 자기 자신을 제외하고 영화를 추천해 주었다.
